In [1]:
import numpy as np
import progressbar

In [3]:
# honest network delay over next n blocks.
def vectorDelayHonest(ps, es, init_endorsers = 24, delay_priority = 40, delay_endorse = 8):
    return (60 * len(ps)
           + delay_priority * sum(ps) 
           + sum([delay_endorse * max(init_endorsers - e, 0) for e in es]))

# attacking network delay over next n blocks.
def vectorDelayAttacker(ps, es, init_endorsers = 24, delay_priority = 40, delay_endorse = 8):
    return (60 * len(ps) 
           + delay_priority * sum(ps) 
           + sum([delay_endorse * max(init_endorsers - e, 0) for e in es[1:]]))

# efficient sample generation
def getAH(alpha):
    x = np.random.geometric(1-alpha)
    if x == 1:
        h = 0
        a = np.random.geometric(alpha)
    else:
        a = 0
        h = x - 1
    return [a, h]

def rewardBlock(p, e):
    if p == 0:
        return e * 1.25
    return e * 0.1875

def rewardEndorsement(p):
    if p == 0:
        return 1.25
    return 0.8333333

def calcHonestSingle(p, e):
    if p == 0:
        return rewardBlock(0, 32) + e * rewardEndorsement(0)
    return e * rewardEndorsement(0)

def calcAttackSingle(p, e):
    return rewardBlock(p, e) + e * rewardEndorsement(p)

def vectorRewardHonest(ps, es):
    totalReward = 0
    for i in range(len(ps)):
        totalReward += calcHonestSingle(ps[i], es[i])
    return totalReward

def vectorRewardAttack(ps, es):
    totalReward = calcAttackSingle(ps[0], 32)
    for i in range(1,len(ps)):
        totalReward += calcAttackSingle(ps[i], es[i])
    return totalReward

def calcCosts(ps, es):
    return vectorRewardHonest(ps, es) - vectorRewardAttack(ps, es)

In [ ]:
def getProbSelfish(alpha, length, sample_size = int(1e5), init_endorsers = 24, delay_priority = 40, delay_endorse = 8):
    bar = progressbar.ProgressBar()
    feasible_count = 0
    for _ in bar(range(sample_size)):
        aVals = []
        hVals = []
        for i in range(length):
            a, h = getAH(alpha)
            aVals.append(a)
            hVals.append(h)
        eVals = np.random.binomial(32, alpha, size = length)
        honest_delay = vectorDelayHonest(hVals, 32 - eVals)
        selfish_delay = vectorDelayAttacker(aVals, eVals)
        if selfish_delay <= honest_delay:
            feasible_count += 1
    return feasible_count / sample_size